#  Trabajo Práctico 2: Análisis con Random Forest - Organización de Datos

**Alumnos y Padrón**  
* Grassano, Bruno - 103855  
* Romero, Adrián   - 103371

https://github.com/brunograssano/TP-Organizacion-de-datos

## Importamos las bibiliotecas que utilizaremos a lo largo del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import prepararSetDeHoldout
from preprocessing import prepararSetDeValidacion
from preprocessing import arbolDeDecisionPreprocessing

In [ ]:
from funcionesAuxiliares import mostrarAUCScore
from funcionesAuxiliares import mostrarROCCurve
from funcionesAuxiliares import mostrarMatrizDeConfusion
from funcionesAuxiliares import escribirPrediccionesAArchivo

## Importamos los datos y los procesamos

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')
X = prepararSetDeDatos(X)
y = prepararSetDeValidacion(y)

In [ ]:
nombres,X_rf = arbolDeDecisionPreprocessing(X) #CREAR RF PREPROCESSING

## Funciones Auxiliares

In [ ]:
def obtenerMejoresParametros(datosPreprocesados):
    mejor_valor = 0
    mejor_profundidad = None
    mejor_cantidad_estimadores = None
    mejor_criterio = None
    y_array=np.array(y)
    for profundidad in [1,2,3,4,5,6,7,9,10]:
        for criterio in ["gini", "entropy"]:
            for cantidad_estimadores in [1,10,50,100,200]:
                kf = StratifiedKFold(n_splits=7)
                metricas = []
                for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
                    rf = RandomForestClassifier(criterion=criterio, max_depth=profundidad, n_estimators=cantidad_estimadores)
                    rf.fit(datosPreprocesados[train_index], y_array[train_index].ravel())
                    predicciones = rf.predict(datosPreprocesados[test_index])
                    score_obtenida = roc_auc_score(y_array[test_index],predicciones)
                    metricas.append(score_obtenida)

                if np.mean(metricas) >= mejor_valor:
                    mejor_valor = np.mean(metricas)
                    mejor_profundidad = profundidad
                    mejor_criterio = criterio
                    mejor_cantidad_estimadores = cantidad_estimadores
            
    return mejor_valor, mejor_profundidad, mejor_criterio, mejor_cantidad_estimadores

### Dividimos el set de datos en sets de training y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rf, y, test_size=0.25, random_state=0)

# Random Forest

Random forest es un ensamble que consiste en el entrenamiento de varios clasificadores de árbol de decisión. 

Cada árbol del ensamble es construido a partir de una muestra simple con reposición del set de entrenamiento. Además se intenta forzar heterogeneidad entre los árboles al escoger el nodo principal de cada árbol a partir de un subconjunto aleatorio de los features. 

En la implementación de sci-kit learn, cada árbol indica la probabilidad de un feature de pertenecer a una clase. Para la predicción final se promedian estas probabilidades. 

Los parámetros que se deben definir son:

**Profundidad máxima:** le impone un limite a la profundidad de máxima de cada árbol, los árboles muy profundos tienden a overfittear.

**Criterio:** es el criterio con el que se escoge el "mejor feature" para ocupar un nodo. Decimos que un feature es mejor que otro si maximiza la ganancia de información o si tiene baja impureza segun el criterio de impureza Gini.

**Cantidad de estimadores:** es la cantidad de árboles que se entrenarán.

In [ ]:
mejor_valor, mejor_profundidad, mejor_criterio, mejor_cantidad_estimadores = obtenerMejoresParametros(X_rf)

In [ ]:
print(f"El mejor valor fue de AUC fue: {mejor_valor}")
print(f"La profundidad encontrada que maximiza el AUC fue: {mejor_profundidad}")
print(f"El criterio encontrado que maximiza el AUC fue: {mejor_criterio}")
print(f"La cantidad de estimadores que maximizan el AUC fue: {mejor_cantidad_estimadores}")

### Evaluamos las métricas

In [ ]:
rf = RandomForestClassifier(max_depth=mejor_profundidad, random_state=0, n_estimators = mejor_cantidad_estimadores, criterion=mejor_criterio)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

### Matriz de confusión 

In [ ]:
mostrarMatrizDeConfusion(y_pred,y_test)

Se puede ver que el random forest entrenado tiene un muy buen accuracy pues lo valores en la diagonal principal son mucho mayores que los de la diagonal invertida.

Se puede observar que tenemos un muy buen precision tambien: de los que predecimos que volverian (55 + 6), efectivamente 55 vuelven, un 90% de precision. Por otro lado el recall no es tan bueno, de los 55 + 25 que volverian pudimos detectar que 55 volverian. Es decir que casi dos tercios de los que volverian los clasificamos correctamente.

### Graficamos la curva ROC

In [ ]:
mostrarROCCurve(rf,"Random Forest",X_test, X_train, y_test, y_train)

Como era de esperarse, la curva ROC sobre el set de entrenamiento es mas cercana a la ideal que aquella sobre el set de test, sin embargo esta diferencia no es tan grande, ambas curvas son bastante similares y ademas muy buenas, su area bajo la curva debe ser cercano a 1.

In [ ]:
mostrarAUCScore(rf,"Random Forest",X_test, y_test)

## Predicciones sobre el nuevo archivo

Realizamos ahora las predicciones del nuevo archivo entregado.

In [ ]:
holdout = pd.read_csv('Datasets/tp-2020-2c-holdout-cols2.csv')
holdout = prepararSetDeHoldout(holdout)
nombres, holdout_rf = arbolDeDecisionPreprocessing(holdout) 

In [ ]:
predicciones_holdout = rf.predict(holdout_rf)

In [ ]:
escribirPrediccionesAArchivo(rf,"Random Forest")